In [1]:
import xpress as xp

In [ ]:
def my_callback(prob, data, isheurstic, cutoff):

    return

In [ ]:
model = xp.problem()

# Maximize x1 + x2 subject to x1 + x2 <= 5 
x1 = xp.var(name="x1", vartype=xp.integer)
x2 = xp.var(name="x2", vartype=xp.integer)
model.addVariable([x1, x2])

model.addConstraint(x1 + x2 <= 5)
model.setObjective(x1 + x2, sense=xp.maximize)

model.addcbpreintsol(my_callback, None, 3) 

model.solve()